In [1]:
import spacy
import numpy as np
import pandas as pd
import re
import jsonlines
from utils.prepare_rules import create_rules 
from utils.prepare_data import create_prepare_file
from utils.expand_model import expand_model
from utils.chart import show_2_pie,show_bar_model
from utils.table import show_table, show_empty_brand
from tqdm import tqdm
import os.path
from spacy.language import Language
from spacy import displacy
import matplotlib.pyplot as plt
import ru_core_news_sm

In [2]:
BASE_PATH = ''#os.path.dirname(__file__)

DATA= BASE_PATH + 'data/pledges - pledges.csv'
BRANDS=BASE_PATH + 'data/brands.jsonl'
RULES=BASE_PATH + 'data/rules.jsonl'
DATA_PREPARE=BASE_PATH + 'data/pledges_prepare.csv'

BRAND_PATH = BASE_PATH + 'data/brands/'
RULES_PATH = BASE_PATH + 'data/rules/'

### ...Загружаем данные для парсинга и данные по брендам

Данные по брендам это подготовленный о jsonl файл со строками вида

```{"id": "CHERRY", "alias": ["черри","чери","CHERY"], "regex": []}```

Данные для парсинга это полученный от автостата файл, содержащий 10000 строк с текстом про машины

In [3]:
data = pd.read_csv(DATA, delimiter='|')
firts_labels = data.columns.values
np.append(firts_labels, "with_space")

array(['id', 'notificationreferencenumber', 'vehicleproperty_vin',
       'vehicleproperty_description_short', 'nBody', 'nChas',
       'BrandModel', 'BrandModel2', 'BrandModel3', 'Brand', 'Brand2',
       'with_space'], dtype=object)

In [4]:
brands = list(jsonlines.open(BRANDS))
list_brands = [x['id'] for x in brands]

### ...Загружаем данные по моделям


По каждой марке есть словарики для модельного ряда. Например словарь для форда выглядит так:

```{"id": "S-MAX", "alias": [], "regex": []}
{"id": "FOCUS", "alias": ["ФОКУС","FUCUS","FOKUS"], "regex": []}
{"id": "FUSION", "alias": ["ФЬЮЖН"], "regex": []}
{"id": "TRANSIT", "alias": ["Транзит"], "regex": []}
{"id": "P-100", "alias": ["P100"], "regex": []}
{"id": "MONDEO", "alias": ["МОНДЕО"], "regex": []}
{"id": "ESCORT", "alias": [], "regex": []}
{"id": "C-MAX", "alias": [], "regex": []}
{"id": "RANGER", "alias": [], "regex": []}
{"id": "F-250", "alias": ["F250"], "regex": []}
{"id": "F-150", "alias": ["F150"], "regex": []}
{"id": "CARGO", "alias": [], "regex": []}
{"id": "ESCAPE", "alias": [], "regex": []}
{"id": "Explorer", "alias": [], "regex": []}
{"id": "FIESTA", "alias": [], "regex": []}
{"id": "EXCURSION", "alias": [], "regex": []}
{"id": "KUGA", "alias": [], "regex": []}
{"id": "TOURNEO", "alias": [], "regex": []}
{"id": "MAVERICK", "alias": [], "regex": []}
{"id": "GALAXY", "alias": [], "regex": []}
{"id": "IVECO", "alias": [], "regex": []}
{"id": "F550", "alias": ["DAMON DAYBREAK"], "regex": []} 
```


### ...Генерируем правила для моделей и марок и сохраняем их в файлы

Для некоторых марок сейчас не описаны их словари с моделями

In [5]:
rules = create_rules(brands)

# save in file
with jsonlines.open(RULES, mode='w') as writer:
    writer.write_all(rules)
    
for brand in list_brands:
    rules_model = create_rules(brands)
    try:
        models = jsonlines.open(BRAND_PATH + brand +'.jsonl')
        rules_model = create_rules(models, label='MODEL', prefix=brand+'_')
        with jsonlines.open(RULES_PATH + brand + '_rules.jsonl', mode='w') as writer:
            writer.write_all(rules_model)
    except:
        pass
#         print('Для бренда '+ brand+ ' не найден файл с моделями')

### Попытаемся отбить пробелы

Данные приходят довольно шумные - местами отсутствуют пробелы. Поэтому берем все названия брендов и марок (больше 2 букв и нецифровых) и отбиваем их пробелами. Также отбиваем высокочастотные русские слова (год, модель, белый итп) склеенные с цифрами или англ буквами.

In [6]:
list_models = []
for brand in list_brands:
    try:
        reader = jsonlines.open(BRAND_PATH + brand +'.jsonl')
        list_models = list_models + list(reader)
    except:
#         print('Для бренда '+ brand+ ' не найден файл с правилами ' + BRAND_PATH + brand +'.jsonl')
        pass

In [7]:
str_with_space = create_prepare_file(data=data['vehicleproperty_description_short'].values, brands=brands+list_models)

data['with_space'] = str_with_space
data.to_csv(DATA_PREPARE, index=False)

Сохраняем Все что получилось в рабочий файл

In [8]:
data = pd.read_csv(DATA_PREPARE, delimiter=',')

### NER

Теперь размечаем сущности (марки, модели) в каждой строке. Всего 10000 строк.
В зависимости от компьютера исполняющего код время может варьироваться от 30сек до 1мин 20сек.

__внимание__ показатель времени не предел, его скорее всего можно сократить

In [9]:
rules = list(jsonlines.open(RULES))
for brand in list_brands:
    if os.path.exists(RULES_PATH + brand + '_rules.jsonl'):
            file = jsonlines.open(RULES_PATH + brand + '_rules.jsonl')
            rules = rules + (list(file))

In [10]:
colors = {"BRAND": "#aa9cfc", "MODEL": "#fc9ce7", "YEAR": "#9cfcb1"}
options = {"ents": ["BRAND", "MODEL", "YEAR"], "colors": colors}

nlp = ru_core_news_sm.load(exclude=['tok2vec', 'morphologizer', 'parser', 'senter', 'attribute_ruler', 'lemmatizer'])

config = { "overwrite_ents": True }
ruler = nlp.add_pipe("entity_ruler", before="ner", config=config) #.from_disk(RULES_PATH +"all_rules.jsonl", )
rules.append({"label": "YEAR", "pattern": [{"LOWER": {"REGEX": "(19|20)\d{2}"}}], "id": "4-digits"})
rules.append({"label": "YEAR", "pattern": [{"LOWER": {"REGEX": "\D(\d{2})\D"}}], "id": "2-digits"})
ruler.add_patterns(rules)

nlp.add_pipe("expand_model", after="ner")

ents_info = []
html=[]
for article in tqdm(data['with_space']):
    article = str(article)
    doc = nlp(article)
    ents_info.append(doc.user_data)
    html.append(doc if len(doc.ents) == 0 else displacy.render(doc, style="ent", jupyter=False, options=options))
    


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:35<00:00, 279.38it/s]


In [11]:
colors = {"BRAND": "#aa9cfc", "MODEL": "#fc9ce7", "YEAR": "#9cfcb1"}
d = data.iloc[29]['vehicleproperty_description_short']
doc = nlp(article)
print([x.label_ for x in doc.ents])
print(doc.user_data)
displacy.render(doc, style="ent",  options=options)

['BRAND', 'MODEL', 'YEAR']
{'count_brands': 1, 'count_models': 1, 'first_brand': 'lada', 'brands': 'lada', 'models': 'lada_priora', 'years': '2010', 'have_fit_model': True}


In [12]:
droped_column_name =[item for item in data.columns.values if item not in firts_labels]
data = data.drop(droped_column_name, axis=1)


In [13]:
ents_info_df = pd.DataFrame.from_records(ents_info)
data.index.name = 'order'
data = data.join(ents_info_df,on='order')
data = data.assign(HTML = html)
data.index.name= 'id'

### График опознанных брендов и моделей

Далее две круговых диаграммы, первая показывает в скольких строках распознанно брендов.
Возможно есть ошибки, потому как в строках есть название продавца/двигателя, который совпадает с брендом.
Но эту проблему можно минимизировать (проверить 2-3 предыдущих токена и если они равны ООО, ЗАО, двигатель, изготовитель и пр) испключить эту сущность (марку бренд).

На второй диаграмме показано сколько моделей было обнаружено среди строк в которых обнаружен бренд. 
<li>синяя область - это строки в которых найденная модель соответствует модельному ряду бренда.</li>
<li>оранжевая область - это строки в которых найденная модель не бьется с брендом - это скорее всего ошибки</li>
<li>зеленая - это строки где модель не была найдена</li>


__внимание__
<li>показатель найденных брендов может быть увеличен, когда мы сами определим что из строки бренд (я не везде смогла это понять)</li>
<li>показатель найденных моделей может быть значительно увеличен, после создания словарей для всех брендов (см диаграмму ниже)</li>
<li>точность определения модели может быть также повышена путем поиска для каждого бренда только моделей из его модельного ряда</li>

In [14]:
# show_2_pie(data)

Ниже 3 диаграмма - столбики бренды. 

<li>Синий - модели нет</li>
<li>Оранжевый модель не та</li>
<li>Зеленый -модель из модельного ряда</li>


по оси Y количество найденных строк для бренда. Диаграмма разрезана на части чтобы ее было легче смотреть.

Бренды для которых зеленого много - это бренды с подготовленными словарями.

In [15]:
# show_bar_model(data, 0, 40)

In [16]:
# show_bar_model(data, 40, 80)

In [17]:
# show_bar_model(data, 80, 120)

Ниже таблица для распознанной марки Форд (первые 100 строк из 305)

In [18]:
finaly_df = show_table(data=data, brand = 'ford', start=0, end=100, only_empty=False)
finaly_df.style

Count rows  305


,HTML,brands,models,have_fit_model,count_brands,count_models,years
id,,,,,,,
36,автомашина FORD BRAND Форд Фокус MODEL 2011 YEAR года выпуска,ford,ford_focus,True,1,1,2011
113,автомашина FORD BRAND ESCAPE MODEL 2007 YEAR года выпускацвет-серо-голубой,ford,ford_escape,True,1,1,2007
162,Автомобиль легковой Ford BRAND Explorer MODEL 2013 YEAR года выпускадвигатель №КХ BDB01537 цвет черныйкузов Z6FBXXESWBDB01537 государственный регистрационный знак О038ХВ77Свидетельство о регистрации 7711 №454089ПТС 16 НН 267999. Собственник – Вербин А.Ю. Залоговая стоимость - 1 040 000.00 руб.,ford,ford_explorer,True,1,1,2013
167,FORD BRAND ФОРД ФОКУС MODEL 2014 YEAR,ford,ford_focus,True,1,1,2014
233,FORD BRAND ФОРД ФОКУС MODEL 2006 YEAR,ford,ford_focus,True,1,1,2006
413,МаркаМодель FORD BRAND МОНДЕО MODEL Наименование (тип ТС) ЛЕГКОВОЙВ год выпуска 2011 YEAR тип№ двигателя БЕНЗИНОВЫЙTPBA BY02515 кузовкабина X9FDXXEEBDBY02615 цвет темно серый Мощность двигателял.с. 23936 (176) Рабочий объем двигателякуб.см 1999 YEAR Организация- изготовитель ТС (страна) Россия Серия№ТДТПО не оформлялось ПТС 47 НВ№ 862062 от 22.06.2011 года,ford,ford_mondeo,True,1,1,"2011, 1999"
493,автомобиль FORD BRAND EXCURSION MODEL категория D? год выпуска 2001 YEAR государственный номерной знак К 001 УМ 65ПТС 77 ТЕ 089006 цвет кузова черный,ford,ford_excursion,True,1,1,2001
507,FORD BRAND ФОРД ФОКУС MODEL год изготовления ТС: 2011 YEAR Идентификационный номер (VIN) : X9FHXXEEDHBC49684 Наименование (тип ТС) : легковой; Категория ТС: В; Модель№ двигателя: AODA BC49684; Шасси (рама) : отсутствует; Кузов №: X9FHXXEEDHBC49684; Цвет кузова: серебристый; Мощность двигателял.с. (кВт) : 145 (107) ; Рабочий объем двигателякуб.см: 1999 YEAR ; Тип двигателя: бензиновый; Разрешенная максимальная массакг.: 1905 YEAR ; Масса без нагрузкикг: 1362; Организация-изготовитель ТС: ЗАО ФОРД МОТОР КОМПАНИ (РОС,ford,ford_focus,True,1,1,"1905, 2011, 1999"
514,МаркаМодель FORD BRAND МОНДЕО MODEL Наименование (тип ТС) ЛЕГКОВОЙВ год выпуска 2011 YEAR тип№ двигателя БЕНЗИНОВЫЙAOBC BU 29739 кузовкабина X9FDXXEEBDBU29739 цвет черный Мощность двигателял.с. 14552 (107) Рабочий объем двигателякуб.см 1999 YEAR Организация- изготовитель ТС (страна) Россия Серия№ТДТПО не оформлялось ПТС 47 НЕ №745284 от 11.10.2011 года,ford,ford_mondeo,True,1,1,"2011, 1999"


Ниже представлена таблица в которой не найден ни один бренд. Здесь нужно понять что является брендом.

In [19]:
finaly_df = show_empty_brand(data=data, start=0, end=400)
finaly_df.style

Count rows  1706


,HTML,brands,models,have_fit_model,count_brands,count_models
id,,,,,,
0,Автомобиль,,,False,0,0
8,Автомобиль,,,False,0,0
9,bvv,,,False,0,0
65,Марка-Экскаватор Одноковшовый ЭО-2202 Год выпуска 2010 YEAR Заводской № машины (рамы) 00229/808102344 Номер двигателя 511316 Коробка передач № 323993 Основной ведущий мост (мосты) № 622219/0710302-04 Цвет сине-желтый Вид движения колесный Регистрационный знак 22 МС 8730 паспорт самоходной машины ВЕ 416060,,,False,0,0
86,Грузовой-тягач специальный2012 YEAR г.в.гос.номер Х692РН69марка YEAR 470104 на шасси МЕRCEDES-BENZмодель № двиг. ОМ501LА54197600499190шасси WDB9340321L192438 цвет белыймощность двиг. 320 л.с.раб. V MODEL двиг. 11946 куб.см.разрешенная макс. масса 18000 кг.масса без нагрузки 7000 кг.страна Россия,,mercedes-benz_v-class,False,0,1
95,Полуприцеп-топливнозаправщик ППЦТ-176; 2012 YEAR года выпускацвет: белыйПТС №31 НМ 669567 выдан 20.07.2012 адрес местонахождения: г.Владимирмкр.Юрьевецул.Ноябрьскаяд.138.,,,False,0,0
96,Полуприцеп-топливнозаправщик ППЦТ-27; 2012 YEAR года выпускацвет: белыйПТС №31 НМ 669568 выдан 20.07.2012 адрес местонахождения: г.Владимирмкр.Юрьевецул.Ноябрьскаяд.138.,,,False,0,0
97,Автопогрузчик HELI CPQD15 (FG15) ; 2012 YEAR года выпускацвет: оранжевыйТС №233057 выдан 09.12.2012 адрес местонахождения: г.Владимирмкр.Юрьевецул.Ноябрьскаяд.138.,,,False,0,0
108,АВТОМОБИЛЬ-ФУРГОН23210ВБ 207 гшода выпускацвет - белый,,,False,0,0


Дальше пример таблица для бренда scania и строки с не обнаруженными для нее моделями. В ее словаре модельного ряда есть только OMNILINE и IRIZAR. Соответсвенно для повышения распознавания моделей необходимо расширить словарь.

In [20]:
finaly_df = show_table(data=data, brand = 'scania', start=0, end=400)
finaly_df.style

Count rows  58


,HTML,brands,models,have_fit_model,count_brands,count_models,years
id,,,,,,,
48,SCANIA BRAND R124CB6X6HZ3602003 Г.В.ПТС: 77 ТM 063767 от 15/10/2003 г. Цвет белый.,scania,,False,1,0,
202,ГРУЗОВОЙ ТЯГАЧ СЕДЕЛЬНЫЙ SCANIA BRAND P 400 CA6X4HSZ 2013 YEAR года выпуска,scania,,False,1,0,2013
252,SCANIA BRAND G 400 MODEL LA4X2HNA грузовой тягач седельный2012 YEAR года выпуска,scania,mercedes-benz_g-class,False,1,1,2012
303,ИПВ 6730T4 Грузовой самосвал SCANIA BRAND P380 2011 YEAR г.в.,scania,,False,1,0,2011
304,ИПВ 6730Т4 (Грузовой самосвал SCANIA BRAND P380) 2011 YEAR г.в.,scania,,False,1,0,2011
313,Грузовой тягач седельный SCANIA BRAND R400LA4X2HLA,scania,,False,1,0,
314,Грузовой тягач седельный SCANIA BRAND R400LA4X2HLA,scania,,False,1,0,
315,Грузовой тягач седельный SCANIA BRAND R400LA4X2HLA,scania,,False,1,0,
316,Грузовой тягач седельный SCANIA BRAND R400LA4X2HLA,scania,,False,1,0,


## Вопросы

Какой разультат будет необходимым и достаточным?
<li> по времени обработки 10000 строк</li>
<li> по количеству определенных брендов </li>
<li> по количеству определенных марок </li>

Определяя модель, какая точность требуется например у КАМАЗ есть очень очень много цифровых моделей - нужны все вариации или их нужно как-то группировать?
